# Ejemplo de como evaluar el diseño de una antena usando NEC++

## Instalacion de paquetes
Instalacion de pandoc, swig y libtool
Estos paquetes son necesarios para compilar el paquete PyNEC.

In [1]:
!sudo apt-get install pandoc swig libtool

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcmark-gfm-extensions0.29.0.gfm.3 libcmark-gfm0.29.0.gfm.3 pandoc-data
  swig4.0
Suggested packages:
  libtool-doc gcj-jdk texlive-latex-recommended texlive-xetex texlive-luatex
  pandoc-citeproc texlive-latex-extra context wkhtmltopdf librsvg2-bin groff
  ghc nodejs php python ruby libjs-mathjax libjs-katex
  citation-style-language-styles swig-doc swig-examples swig4.0-examples
  swig4.0-doc
The following NEW packages will be installed:
  libcmark-gfm-extensions0.29.0.gfm.3 libcmark-gfm0.29.0.gfm.3 libtool pandoc
  pandoc-data swig swig4.0
0 upgraded, 7 newly installed, 0 to remove and 45 not upgraded.
Need to get 21.8 MB of archives.
After this operation, 163 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcmark-gfm0.29.0.gfm.3 amd64 0.29.0.gfm.3-3 [115 kB]
Get:2 http://archi

Descargamos el repositorio de Python-NECpp

In [2]:
!git clone --recursive https://github.com/tmolteno/python-necpp.git

Cloning into 'python-necpp'...
remote: Enumerating objects: 544, done.
remote: Counting objects: 100% (101/101), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 544 (delta 49), reused 87 (delta 41), pack-reused 443
Receiving objects: 100% (544/544), 137.39 KiB | 11.45 MiB/s, done.
Resolving deltas: 100% (317/317), done.
Submodule 'necpp_src' (https://github.com/tmolteno/necpp.git) registered for path 'necpp_src'
Cloning into '/content/python-necpp/necpp_src'...
remote: Enumerating objects: 3072, done.        
remote: Counting objects: 100% (56/56), done.        
remote: Compressing objects: 100% (39/39), done.        
remote: Total 3072 (delta 20), reused 37 (delta 15), pack-reused 3016        
Receiving objects: 100% (3072/3072), 2.91 MiB | 16.93 MiB/s, done.
Resolving deltas: 100% (2269/2269), done.
Submodule path 'necpp_src': checked out '0af22b10d5b8e96da412009ef1178f1b4131cbb8'


Instalamos PyNEC

In [3]:
%cd python-necpp/PyNEC
!./build.sh

/content/python-necpp/PyNEC
Submodule path '../necpp_src': checked out '870f110875ef7e1fc951cbd489fde20ac9a93284'
libtoolize --force --copy
libtoolize: putting auxiliary files in AC_CONFIG_AUX_DIR, 'm4'.
libtoolize: copying file 'm4/ltmain.sh'
libtoolize: putting macros in AC_CONFIG_MACRO_DIRS, 'config/m4'.
libtoolize: copying file 'config/m4/libtool.m4'
libtoolize: copying file 'config/m4/ltoptions.m4'
libtoolize: copying file 'config/m4/ltsugar.m4'
libtoolize: copying file 'config/m4/ltversion.m4'
libtoolize: copying file 'config/m4/lt~obsolete.m4'
aclocal
autoheader
automake --add-missing
configure.ac:6: installing 'm4/compile'
configure.ac:6: installing 'm4/config.guess'
configure.ac:6: installing 'm4/config.sub'
configure.ac:5: installing 'm4/install-sh'
configure.ac:5: installing 'm4/missing'
src/Makefile.am: installing 'm4/depcomp'
autoconf
checking for a BSD-compatible install... /usr/bin/install -c
checking whether build environment is sane... yes
checking for a race-free mkdi

Finalmente, instalamos el paquete PyNEC en python.

In [4]:
!pip3 install .

Processing /content/python-necpp/PyNEC
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for PyNEC: filename=PyNEC-1.7.3.6-cp310-cp310-linux_x86_64.whl size=2133739 sha256=fbf1704f726fbdd37aaa5dab56b54ae0ec03a491343a8f677be6c2b2cf802562
  Stored in directory: /tmp/pip-ephem-wheel-cache-ijtw9gp3/wheels/c3/55/21/71cf4aa828eef2d605c2d8ce2054dfbf9bcaa9d05b346ce24a
Successfully built PyNEC


## Importar librerias y definir funcion de ganancia
Importamos los paquete que se van a usar en este notebook

In [8]:
from PyNEC import *
import numpy as np
import plotly.express as px

Definimos los parametros para la simulacion de los campos electromagneticos, con la variable `numSegments` controlamos la cantidad de segmentos del problema

In [99]:
thickness = 0.002 #Grosor del alambre en metros
freq = 2500000000 #Frecuencia de resonancia de interes
numSegments = 2   #Numero de segmentos a doblar
waveLength = 299792458.0 / freq #calculo de la longitud de onda
tol = 1e-3        #Tolerancia entre segmentos
directionality = 2 #Seleccionar que funcion optimizar

Definimos la funcion a evaluar, en este caso nos interesa maximizar la ganacia de la antena en la frecuencia de 2.5GHz. Este funcion esta diseñada para 2 segmentos, se debe modificar para incluir otra cantidad de segmentos.

In [109]:
def ObjFunc(params):
    #Params contiene los elementos x1,y1,z1,x2,y2,z2
    x1 = params[0]
    y1 = params[1]
    z1 = params[2]
    x2 = params[3]
    y2 = params[4]
    z2 = params[5]

    #creation of a nec context
    context=nec_context()
    #get the associated geometry
    geo = context.get_geometry()
    #Sometimes the design crashes
    try:
      #add wires to the geometry
      geo.wire(1, 1, 0.0, 0.0, 0.0, x1, y1, z1, thickness, 1.0, 1.0)
      geo.wire(2, 1, x1+tol, y1+tol, z1+tol, x2, y2, z2, thickness, 1.0, 1.0)
      #finish design
      context.geometry_complete(0)
      #Test signals set-up
	    #check: https://pypi.org/project/necpp/
      #Define ground's location
      context.gn_card(1, 0, 0, 0, 0, 0, 0, 0)
      #Define excitation source parameters
      context.ex_card(0, 1, 1, 0, 1.0, 0, 0, 0, 0, 0)
      #Define range of frequencies, this case only 2.5GHz
      context.fr_card(0, 1, freq / 1000000, 0.0)
      #Define radiation pattern parameters
      context.rp_card(0, 17, 45, 0, 5, 0, 0, 0, 0, 5, 8, 0, 0)

      # Objective function output
      if (directionality == 0):
        gain = context.get_gain_max(0)
      elif (directionality==1):
        gain = context.get_gain_min(0)
      else:
        gain = context.get_gain_mean(0)
    except: #If simulation crashes then assign -999 to gain value.
      gain = -999.0
    return gain

Verificamos la correcta ejecucion de la funcion anterior a partir de unos puntos aleatorios que no superen $\lambda$/2.

In [114]:
bestGain = 0
for k in range(1000):
  np.random.seed(k)
  parameters = np.random.rand(3*numSegments)*waveLength/2.0
  gain = ObjFunc(parameters)

  print(k, gain)
  if bestGain<gain:
    gain = bestGain
    bestParams = parameters.copy()

  if gain < -900:
    break

0 0.7075200783750574
1 1.62170765787929
2 0.6540168464739935
3 1.0913462631751696
4 0.3382429043140919
5 1.0728640025512628
6 -1.18317601242824
7 1.173821196257383
8 -0.15472604135016463
9 -0.04385429706289015
10 0.9123077972565694
11 0.858579899972309
12 1.0309919438076403
13 0.6818321110515929
14 -0.08350106075464873
15 2.4969984345920664
16 -0.6203577525056865
17 1.233880369009128
18 0.41957288316477853
19 -0.6833937702141414
20 0.3824073191358803
21 -0.7960407821595604
22 1.1768381538611368
23 -0.49395988747740655
24 -1.2394021188741329
25 -1.0706496539927062
26 0.518005714327042
27 0.6554671482046373
28 1.380502577417843
29 1.5932841129570872
30 0.929533659827683
31 0.34831436011637373
32 0.9126226612815653
33 1.2028040994304472
34 1.1303097648602185
35 1.2606122608446222
36 0.46070238283588183
37 1.0145980484880737
38 0.32864778140277034
39 -0.13767422532060306
40 0.012947156264510533
41 -0.7381716234407062
42 0.24682156922431758
43 0.31743636252734875
44 0.2734033952727093
45 0.

Tomamos los parametros y los volemos una matriz de 3x3 (numSegments+1,3). Lo hacemos con el de hacer mas facil el grafico de la antena.

In [116]:
x3D = bestParams.copy()
x3D = np.insert(x3D, 0 , [0.,0.,0.])
matriz = x3D.reshape(numSegments+1,3)
xline = matriz[:,0]
yline = matriz[:,1]
zline = matriz[:,2]

Finalmente graficamos el diseño de la antena

In [117]:
fig = px.line_3d(x=xline, y=yline, z=zline)
fig.show()